In [16]:
%load_ext autoreload
%autoreload 2

from jaref_bot.data.http_api import ExchangeManager, BybitRestAPI, OKXRestAPI, GateIORestAPI
from jaref_bot.db.postgres_manager import DBManager
from jaref_bot.db.redis_manager import RedisManager
from jaref_bot.strategies.arbitrage import find_tokens_to_close_order, get_close_volume
from jaref_bot.trading.trade_api import BybitTrade, OkxTrade, GateTrade
from jaref_bot.utils.coins import get_step_info, get_min_qty, round_volume
from jaref_bot.data.data_functions import is_data_up_to_date
from jaref_bot.trading.functions import place_limit_order, cancel_order, set_leverage


from jaref_bot.config.credentials import host, user, password, db_name

import signal
from IPython.display import clear_output

import pandas as pd
import polars as pl
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import logging
from datetime import datetime
from time import sleep
import pickle
from copy import deepcopy
from decimal import Decimal, ROUND_DOWN, getcontext
getcontext().prec = 8

import redis
from redis.exceptions import ConnectionError

from asyncio.exceptions import CancelledError
import aiohttp, asyncio
import nest_asyncio
nest_asyncio.apply()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s")
logging.getLogger('aiohttp').setLevel('ERROR')
logging.getLogger('asyncio').setLevel('ERROR')
logger = logging.getLogger()

In [18]:
def signal_handler(sig, frame):
    global exit_flag
    print('Завершение работы.')
    exit_flag = True

In [19]:
def handle_close_position(demo, resp, exc, symbol, order_type):
    exc_dict = {'bybit': BybitTrade, 'okx': OkxTrade, 'gate': GateTrade}
    exchange, market_type = exc.split('_')
    exc_manager = exc_dict[exchange](demo=demo)

    ct_val = coin_information[exc][symbol]['ct_val']

    if exchange == 'gate':
        handle_gate_resp(resp=resp, market_type=market_type)
    else:
        pos = exc_manager.get_order(symbol=symbol, market_type='linear', order_id=resp, order_type=order_type, ct_val=ct_val)
        if pos:
            postgre_manager.close_order(token=symbol, exchange=exchange, market_type=market_type, qty=abs(pos['qty']), close_price=pos['price'], 
                               close_usdt_amount=pos['usdt_amount'], close_fee=pos['fee'])
            logger.info(f'[POSITION CLOSE] {pos['order_side']} {pos['qty']} {symbol} for {pos['usdt_amount']} (price: {pos['price']}) on {exchange}')

In [20]:
def handle_gate_resp(resp, market_type):
    _id = resp['id']
    fee_proc = Decimal(resp['tkfr'])
    token = resp['contract']
    price = Decimal(resp['fill_price'])
    contr_size = Decimal(abs(resp['size']))
    
    ct_val = coin_information['gate' + '_' + market_type][token]['ct_val']
    qty = contr_size * ct_val
    usdt_amount = (qty * price).normalize()
    side = 'buy' if resp['size'] > 0 else 'sell'
    fee = (fee_proc * usdt_amount).normalize()
    

    postgre_manager.close_order(token=token, exchange='gate', market_type=market_type, qty=qty, close_price=price, 
                           close_usdt_amount=usdt_amount, close_fee=fee)
    logger.info(f'[POSITION CLOSE] {side} {qty} {token} for {usdt_amount} (price: {price}) on gate')

In [21]:
market_fees = {'bybit_spot': 0.001, 'bybit_linear': 0.001, 'okx_spot': 0.001, 'okx_linear': 0.0005,
               'gate_spot': 0.002, 'gate_linear': 0.0005}

In [22]:
logger.info('Инициализируем биржи...')
# ====================================
# Инициация нужных криптобирж, рынков и БД
exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
exc_manager.add_market("okx_linear", OKXRestAPI('linear'))
exc_manager.add_market("gate_linear", GateIORestAPI('linear'))

db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
postgre_manager = DBManager(db_params)

redis_client = redis.Redis(db=0, decode_responses=True)
redis_price_manager = RedisManager(db_name = 'orderbooks')
redis_order_manager = RedisManager(db_name = 'orders')

try: 
    redis_client.ping()
    print('Сервер Redis запущен')
except ConnectionError:
    print('Сервер Redis не отвечает')

2025-04-14 09:44:38 [INFO] Инициализируем биржи...


Сервер Redis запущен


In [23]:
with open("./data/coin_information.pkl", "rb") as f:
    coin_information = pickle.load(f)

# with open("./data/close_order_history.pkl", "rb") as f:
#     orders_history = pickle.load(f)

In [25]:
logger.info('Запускаем основной цикл программы...')

close_tokens = {'ADA_USDT': {'long_exc': 'bybit', 'short_exc': 'okx', 'min_edge': -0.045},
                'ALGO_USDT': {'long_exc': 'bybit', 'short_exc': 'gate', 'min_edge': -0.016}
                    }

exit_flag = False
signal.signal(signal.SIGINT, signal_handler)
# ====================================
# Параметры запуска
limit_type = 'limit' # Первый ордер ставим по лучшей цене, а второй отправляем сразу же, как сматчился первый
n_ticks = Decimal('2')

demo = True
leverage = 1
max_usdt_order = 100
min_usdt_order = 50

max_orders = 3
# ====================================
# Основной цикл
while not exit_flag:
    token, data = None, None
    token_dict = dict()

    current_orders = postgre_manager.get_table('current_orders')
    n_current_orders = len(current_orders) // 2
    pending_orders = redis_order_manager.get_pending_orders()
    
    try:
        # Скачиваем текущие данные по ордербукам из Redis
        try:
            current_data = redis_price_manager.get_orderbooks(n_levels=5).filter(pl.col('symbol').is_in(arbitrage_tokens))
        except pl.exceptions.ColumnNotFoundError:
            sleep(1)
            continue

        # Проверка на то, что данные являются актуальными, что ни одна из бирж не подвисла
        if not is_data_up_to_date(current_data, time_thresh=10):
            logger.error(f'Устаревшие данные по ценам.')
            sleep(1)
        if len(current_data) == 0:
            raise Exception('No data!')

        sleep(0.5)
    except (KeyboardInterrupt, CancelledError):
        print('Завершение работы.')
        break

2025-04-14 09:50:57 [INFO] Запускаем основной цикл программы...


Завершение работы.
Завершение работы.


In [ ]:
# Добавить проверку на профит при закрытии сделки.

# Добавить учёт фандинга в профит
# Посмотреть, чего за хуйня с определением профита на gate.io. Можно запросить ордеры по _id. История есть в окне трейдинга.

#### Закрытие арбитражных ордеров

In [ ]:
# logger.info('Запускаем основной цикл программы...')

# demo = False

# bybit_trade = BybitTrade(demo=demo)
# okx_trade = OkxTrade(demo=demo)
# gate_client = GateTrade(demo=demo)

# std_coef = 1.8
# min_edge = -0.05

# exit_flag = False
# signal.signal(signal.SIGINT, signal_handler)

# while not exit_flag:
#     token, data = None, None
#     close_df = pd.DataFrame()
#     token_list = []
#     res = dict()
    
#     try:
#         stats_data = postgre_manager.get_table('stats_data')
#         stats_data = pl.from_pandas(stats_data)
#         current_orders = postgre_manager.get_table('current_orders')
#         current_orders = pl.from_pandas(current_orders)

#         if len(current_orders) == 0:
#             sleep(10)
#             continue
        
#         # Скачиваем текущие данные по ордербукам из Redis
#         try:
#             current_data = redis_manager.get_orderbooks(n_levels=5)
#         except pl.exceptions.ColumnNotFoundError:
#             sleep(1)
#             continue
        
#         # Проверка на то, что данные являются актуальными, что ни одна из бирж не подвисла
#         sorted_df = current_data.sort(by='ts')
#         oldest_ts_exc = sorted_df.select("exchange").head(1).item()
#         last_entry = sorted_df.filter(pl.col('exchange') == oldest_ts_exc).select("ts").tail(1).item()
#         time_delta = int(datetime.timestamp(datetime.now())) - last_entry
        
#         # Если данные на бирже последний раз обновлялись больше 20 секунд назад, прерываем работу программы
#         if time_delta > 10:
#             logger.error(f'Отсутствует соединение с биржей {oldest_ts_exc}.')
#             sleep(5)
#             continue
        
#         df_out = find_tokens_to_close_order(current_data=current_data, 
#                                             current_orders=current_orders, 
#                                             stats_data=stats_data, 
#                                             std_coef=std_coef, 
#                                             min_edge=min_edge)
#         if df_out is None:
#             sleep(0.05)
#             continue
        
#         for row in df_out.to_dicts():
#             token = row["token"]
#             long_exc = row["long_exc"]
#             short_exc = row["short_exc"]

#             long_exc_full = long_exc + '_linear'
#             short_exc_full = short_exc + '_linear'
#             qty_remain = float(row['qty'])

#             lm_profit = row["lm_profit"]
#             sm_profit = row["sm_profit"]
#             profit = row["profit"]
        
#             mean = row['cmean']
#             std = row['cstd']
#             diff_edge = mean + std_coef * std
        
#             min_qty = get_min_qty(coin_information, token, long_exc=long_exc_full, short_exc=short_exc_full)
#             qty_step = get_step_info(coin_information, token, long_exc=long_exc_full, short_exc=short_exc_full)
        
#             df_long = current_data.filter((pl.col("exchange") == long_exc) & (pl.col("symbol") == token))
#             df_short = current_data.filter((pl.col("exchange") == short_exc) & (pl.col("symbol") == token))
            
#             long_price = df_long.select('bidprice_0', 'bidprice_1', 'bidprice_2', 'bidprice_3', 'bidprice_4').row(0)
#             long_size = df_long.select('bidvolume_0', 'bidvolume_1', 'bidvolume_2', 'bidvolume_3', 'bidvolume_4').row(0)
#             short_price = df_short.select('askprice_0', 'askprice_1', 'askprice_2', 'askprice_3', 'askprice_4').row(0)
#             short_size = df_short.select('askvolume_0', 'askvolume_1', 'askvolume_2', 'askvolume_3', 'askvolume_4').row(0)
#             long_ob = [[price, size] for price, size in zip(long_price, long_size)]
#             short_ob = [[price, size] for price, size in zip(short_price, short_size)]
            
#             edge = max(diff_edge, min_edge)
#             res = get_close_volume(long_ob, short_ob, diff_edge=edge, min_qty=min_qty, max_qty=qty_remain)

#             print(row)
#             print(f'long: {long_ob}')
#             print(f'short: {short_ob}')
#             print('==========')
            
#             if res and res.get('volume', 0) >= min_qty:
#                 volume = round_volume(volume=res['volume'], qty_step=qty_step)

#                 # Если при округлении цена упала меньше минимально возможной
#                 if volume < min_qty:
#                     continue
                
#                 ct = datetime.now().strftime('%H:%M:%S')
#                 curr_diff = res['edge']
#                 deviation = (float(curr_diff) - mean) / std
                
#                 long_price = long_ob[0][0]
#                 short_price = short_ob[0][0]

#                 orders_history.append({'time': ct, 'row': row, 'long_ob': long_ob, 'short_ob': short_ob, 
#                                        'res': res, 'min_qty': min_qty, 'qty_step': qty_step,
#                                       'lm_profit': lm_profit, 'sm_profit': sm_profit, 'profit': profit})

#                 with open("./data/close_order_history.pkl", "wb") as f:
#                     pickle.dump(orders_history, f)
                
#                 logger.info(f'[PLACE CLOSE ORDER] {volume} {token}; diff: {curr_diff:.3f}, {diff_edge=:.3f}, {mean=:.3f}, {std=:.3f}; {deviation=:.3f}')
#                 logger.info(f'Цены во время простановки ордера. long: {long_price} ({long_exc}), short: {short_price} ({short_exc})')

#                 # Закрываем лонг-ордер
#                 long_order_resp = place_market_order(demo=demo, exc=long_exc_full, symbol=token, side='sell', volume=volume)
#                 logger.info(f'Ордер на лонг-рынке ({long_exc}) отправлен на биржу.')
                
#                 # Закрываем шорт-ордер
#                 short_order_resp = place_market_order(demo=demo, exc=short_exc_full, symbol=token, side='buy', volume=volume)
#                 logger.info(f'Ордер на шорт-рынке ({short_exc}) отправлен на биржу.')
                
#                 # Обрабатываем полученные ответы и заносим ордеры в БД
#                 handle_close_position(demo=demo, resp=long_order_resp, exc=long_exc_full, symbol=token, order_type='market')
#                 handle_close_position(demo=demo, resp=short_order_resp, exc=short_exc_full, symbol=token, order_type='market')

#         sleep(0.05)
#     except (KeyboardInterrupt, CancelledError):
#         print('Завершение работы.')
#         sys.exit()
#     except RuntimeError as e:
#         print(f"Ошибка выполнения: {e}")